In [103]:
import pandas as pd
import numpy as np

In [104]:
df = pd.read_csv("input/full_RNAseq_LRT_added_mean_added.csv")

In [105]:
gene_list_df = pd.read_csv("input/Tfh_specific_genes_Immunity2014_v2.csv")

In [106]:
gene_l = gene_list_df['X'].tolist()

In [107]:
new_l = []
for item in df['Unnamed: 0']:
    if item in gene_l:
        new_l.append(item)
    else:
        new_l.append("NA")

In [108]:
df['annot'] = new_l

In [109]:
df.to_csv("input/full_RNAseq_LRT_added_mean_added_annot.csv",index=False)

-------------

# anootate pathway
* mitochondria

In [110]:
import pandas as pd

In [111]:
oxphos_gl = pd.read_csv("input/gl_KEGG_OXIDATIVE_PHOSPHORYLATION_4HTprep.csv")
TCA_gl = pd.read_csv("input/gl_KEGG_CITRATE_CYCLE_TCA_CYCLE_4HTprep.csv")
ribosome_gl = pd.read_csv("input/gl_KEGG_RIBOSOME_HT4R.csv")

In [112]:
oxphos_gl['metabolism'] = 'OXPHOS'
TCA_gl['metabolism'] = 'TCA_cycle'
ribosome_gl['metabolism'] = 'Ribosome'

In [113]:
gl_df = pd.concat([oxphos_gl,TCA_gl,ribosome_gl],axis =0)

In [114]:
gl_df.shape

(205, 2)

In [115]:
a_list = gl_df['mouseSymbol'].tolist()

In [116]:
duplicate_gl = []
for item in a_list:
    if a_list.count(item) > 1:
        duplicate_gl.append(item)

In [117]:
uni_gl = [x for x in set(gl_df['mouseSymbol']) if x not in duplicate_gl]

In [118]:
uni_df = gl_df.loc[gl_df['mouseSymbol'].isin(uni_gl),:]

In [119]:
duplicate_gl = list(set(duplicate_gl))

In [120]:
dup_df_cor = pd.DataFrame({"mouseSymbol": duplicate_gl, "metabolism": ["OXPHOS"]*len(duplicate_gl)})

In [121]:
df_met_annot = pd.concat([uni_df,dup_df_cor],axis=0)

In [122]:
len(set(df_met_annot['mouseSymbol']))

201

In [123]:
df.columns

Index(['Unnamed: 0', 'B6_Tfh_mean', 'B6_Tn_mean', 'TC_Tfh_mean', 'TC_Tn_mean',
       'B6.1.Tfh', 'B6.2.Tfh', 'B6.3.Tfh', 'B6.4.Tfh', 'B6.1.Tn', 'B6.2.Tn',
       'B6.3.Tn', 'B6.4.Tn', 'TC1.Tfh', 'TC2.Tfh', 'TC4.Tfh', 'TC1.Tn',
       'TC2.Tn', 'TC4.Tn', 'log2FoldChange_TCTfhvsTCTn', 'stat_TCTfhvsTCTn',
       'pvalue_TCTfhvsTCTn', 'padj_TCTfhvsTCTn', 'log2FoldChange_B6TfhvsB6Tn',
       'stat_B6TfhvsB6Tn', 'pvalue_B6TfhvsB6Tn', 'padj_B6TfhvsB6Tn',
       'log2FoldChange_interaction', 'stat_interaction', 'pvalue_interaction',
       'padj_interaction', 'log2FoldChange_TCTfhvsB6Tfh', 'stat_TCTfhvsB6Tfh',
       'pvalue_TCTfhvsB6Tfh', 'padj_TCTfhvsB6Tfh', 'log2FoldChange_TCTnvsB6Tn',
       'stat_TCTnvsB6Tn', 'pvalue_TCTnvsB6Tn', 'padj_TCTnvsB6Tn',
       'log2FoldChange_LRT', 'stat_LRT', 'pvalue_LRT', 'padj_LRT', 'annot'],
      dtype='object')

In [124]:
df2 = pd.merge(df,df_met_annot,left_on = "Unnamed: 0",right_on = "mouseSymbol",how = "left")

# annotate statistics
* co-up general 
* co-up FC1.5
* co-down general
* co-down FC1.5

In [125]:
df2['stat_annot'] = np.nan

In [126]:
df2.loc[(df2['padj_TCTnvsB6Tn'] < 0.05)
        & (df2['padj_TCTfhvsB6Tfh'] < 0.05)
        & (df2['log2FoldChange_TCTfhvsB6Tfh'] > 0) 
        & (df2['log2FoldChange_TCTnvsB6Tn'] > 0),"stat_annot"] = "co_high_FC0"

In [127]:
df2.loc[(df2['padj_TCTnvsB6Tn'] < 0.05)
        & (df2['padj_TCTfhvsB6Tfh'] < 0.05)
        & (df2['log2FoldChange_TCTfhvsB6Tfh'] < 0) 
        & (df2['log2FoldChange_TCTnvsB6Tn'] < 0),"stat_annot"] = "co_low_FC0"

In [128]:
df2.loc[(df2['padj_TCTnvsB6Tn'] < 0.05)
        & (df2['padj_TCTfhvsB6Tfh'] < 0.05)
        & (df2['log2FoldChange_TCTfhvsB6Tfh'] > 0.5850) 
        & (df2['log2FoldChange_TCTnvsB6Tn'] > 0.5850),"stat_annot"] = "co_high_FC1.5"

In [129]:
df2.loc[(df2['padj_TCTnvsB6Tn'] < 0.05)
        & (df2['padj_TCTfhvsB6Tfh'] < 0.05)
        & (df2['log2FoldChange_TCTfhvsB6Tfh'] < -0.5850) 
        & (df2['log2FoldChange_TCTnvsB6Tn'] < -0.5850),"stat_annot"] = "co_low_FC1.5"

--------

# Add interaction term

In [130]:
df2.columns

Index(['Unnamed: 0', 'B6_Tfh_mean', 'B6_Tn_mean', 'TC_Tfh_mean', 'TC_Tn_mean',
       'B6.1.Tfh', 'B6.2.Tfh', 'B6.3.Tfh', 'B6.4.Tfh', 'B6.1.Tn', 'B6.2.Tn',
       'B6.3.Tn', 'B6.4.Tn', 'TC1.Tfh', 'TC2.Tfh', 'TC4.Tfh', 'TC1.Tn',
       'TC2.Tn', 'TC4.Tn', 'log2FoldChange_TCTfhvsTCTn', 'stat_TCTfhvsTCTn',
       'pvalue_TCTfhvsTCTn', 'padj_TCTfhvsTCTn', 'log2FoldChange_B6TfhvsB6Tn',
       'stat_B6TfhvsB6Tn', 'pvalue_B6TfhvsB6Tn', 'padj_B6TfhvsB6Tn',
       'log2FoldChange_interaction', 'stat_interaction', 'pvalue_interaction',
       'padj_interaction', 'log2FoldChange_TCTfhvsB6Tfh', 'stat_TCTfhvsB6Tfh',
       'pvalue_TCTfhvsB6Tfh', 'padj_TCTfhvsB6Tfh', 'log2FoldChange_TCTnvsB6Tn',
       'stat_TCTnvsB6Tn', 'pvalue_TCTnvsB6Tn', 'padj_TCTnvsB6Tn',
       'log2FoldChange_LRT', 'stat_LRT', 'pvalue_LRT', 'padj_LRT', 'annot',
       'mouseSymbol', 'metabolism', 'stat_annot'],
      dtype='object')

In [137]:
df2['stat_interaction'] = np.nan
df2.loc[((df2['padj_interaction'] < 0.05) & (df2["stat_annot"] == "co_high_FC1.5"))|
        ((df2['padj_interaction'] < 0.05) & (df2["stat_annot"] == "co_low_FC1.5")), "stat_interaction"] = "same_dir"

df2.loc[((df2['padj_interaction'] < 0.05) & (df2["stat_annot"] == "co_high_FC0"))|
        ((df2['padj_interaction'] < 0.05) & (df2["stat_annot"] ==  "co_low_FC0")), "stat_interaction"] = "semi_same_dir"

df2.loc[((df2['padj_interaction'] < 0.05)  & 
        (df2['padj_TCTfhvsTCTn'] < 0.05) & 
        (df2['padj_B6TfhvsB6Tn'] < 0.05) & 
        (df2['log2FoldChange_TCTfhvsTCTn'] > 0) &
        (df2['log2FoldChange_B6TfhvsB6Tn'] < 0))|
        
        ((df2['padj_interaction'] < 0.05)  & 
        (df2['padj_TCTfhvsTCTn'] < 0.05) & 
        (df2['padj_B6TfhvsB6Tn'] < 0.05) & 
        (df2['log2FoldChange_TCTfhvsTCTn'] < 0) &
        (df2['log2FoldChange_B6TfhvsB6Tn'] > 0)) ,"stat_interaction"] = "diff_dir"


In [138]:
df2.to_csv("input/full_RNAseq_LRT_added_mean_added_annot_meta_stat_v3.csv",index=False)